## Εξίσωση αντίδρασης - διάχυσης
________


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

plt.style.use("default")
plt.rcParams["figure.figsize"] = [5, 4]  # [width_inches, height_inches]
plt.rcParams["animation.html"] = "jshtml"

### Εξίσωση αντίδρασης - διάχυσης

[Reaction-diffusion equation](https://en.wikipedia.org/wiki/Reaction%E2%80%93diffusion_system)

Αποτελεί ένα μαθηματικό μοντέλο το οποίο περιγράφει τις μεταβολές ως προς τον \
χρόνο και τον χώρο φυσικών ποσοτήτων διαφόρων κλάδων, κυρίως της χημείας \
και της βιολογίας.

Για παράδειγμα, η εξίσωση αντίδρασης-διάχυσης χρησιμοποιείται για την \
προσομοίωση της συγκέντρωσης χημικών ουσιών ή της πυκνότητας κυττάρων.

Μοντέλο Gray-Scott

$$
\begin{align*}
& \frac{\partial u}{\partial t} = D_u \nabla^2 u - uv^2 + F(1-u) \\[20pt]
& \frac{\partial v}{\partial t} = D_v \nabla^2 v + uv^2 - (F+K)v
\end{align*}
$$

&nbsp;

$$
\begin{align*}
& U + 2V \rightarrow 3V \\[2pt]
& V \rightarrow P
\end{align*}
$$


- $U, V, P$: τα χημικά είδη (chemical species)

- $u, v$: οι συγκεντρώσεις των $U$, $V$

- $D_u, D_v$: οι συντελεστές διάχυσης

- $F, K$: οι ρυθμοί αντίδρασης (πόσο γρήγορα ένα είδος δημιουργείται ή εξαφανίζεται)

- $uv^2$: η πιθανότητα της αντίδρασης

Ο λαπλασιανός τελεστής προσεγγίζεται σε αυτό το πρόβλημα αριθμητικά ως [εξής](https://en.wikipedia.org/wiki/Discrete_Laplace_operator#Implementation_via_operator_discretization):

$$
\nabla^2 u^n_{i,j} \approx u^n_{i,j-1} + u^n_{i-1,j} -4u^n_{i,j} + u^n_{i+1, j} + u^n_{i, j+1}
$$

$$
\nabla^2 v^n_{i,j} \approx v^n_{i,j-1} + v^n_{i-1,j} -4v^n_{i,j} + v^n_{i+1, j} + v^n_{i, j+1}
$$

### Δημιουργία των σταθερών του προβλήματος

In [ ]:
Du = 0.16
Dv = 0.08
F = 0.035
K = 0.060

### Δημιουργήστε τον x-άξονα

$0 \leq x \leq 1$

$\delta x = h_x = 0.004$

Χρησιμοποιήστε τη συνάρτηση `np.linspace`.

In [ ]:
x0 = 0
xN = 1
hx = 0.004
Nx = int((xN - x0) / hx + 1)
x = np.linspace(start=x0, stop=xN, num=Nx, endpoint=True, retstep=False)


### Δημιουργήστε τον y-άξονα

$0 \leq y \leq 1$

$\delta y = h_y = 0.004$

Χρησιμοποιήστε τη συνάρτηση `np.linspace`.

In [ ]:
y0 = 0
yN = 1
hy = 0.004
Ny = int((yN - y0) / hy + 1)
y = np.linspace(start=y0, stop=yN, num=Ny, endpoint=True, retstep=False)


### Δημιουργήστε τον t-άξονα

$\delta t = k = 1$

Χρησιμοποιήστε τη συνάρτηση `np.arange` και 3000 χρονικές στιγμές (`Nt=3000`).

In [ ]:
t0 = 0
k = 1

Nt = 3000
tN_plus_k = Nt * k
t = np.arange(t0, tN_plus_k, k)


### Δημιουργήστε κενούς 3-D πίνακες για τα $u, v$

In [ ]:
u = np.full((Nt, Nx, Ny), np.nan)
v = np.full((Nt, Nx, Ny), np.nan)


### Δημιουργήστε το πλέγμα του προβλήματος


Χρησιμοποιήστε τους δύο άξονες (x, y) και τη συνάρτηση `np.meshgrid`

Να θέσετε την παράμετρο `indexing` του `np.meshgrid` ίση με `"ij"` έτσι ώστε να \
τοποθετηθούν με σωστή σειρά οι $x$ και $y$ άξονες.

In [ ]:
xx, yy = np.meshgrid(x, y, indexing="ij")


### Αρχικές συνθήκες

$u = 1,$ εκτός από:

$\;0.4 < x < 0.6\;$ και $\;0.4 < y < 0.6,\;$ όπου έχουμε $u = 0.5\;$

&nbsp;

$v = 0,$ εκτός από:

$\;0.4 < x < 0.6\;$ και $\;0.4 < y < 0.6,\;$ όπου έχουμε $v = 0.25\;$


In [ ]:
u[0] = 1
u[0, (0.4<xx) & (xx<0.6) & (0.4<yy) & (yy<0.6)] = 0.50


### Εισάγετε τη δεύτερη αρχική συνθήκη

In [ ]:
v[0] = 0
v[0, (0.4<xx) & (xx<0.6) & (0.4<yy) & (yy<0.6)] = 0.25


### 2-D Διάγραμμα

### Σχεδιάστε την αρχική κατάσταση της μεταβλητής $u$

In [ ]:
fig, ax = plt.subplots()
plt.close()

ax.imshow(u[0], vmin=0, vmax=1)
fig


### Αριθμητική λύση

Διερευνήστε την αριθμητική λύση του προβλήματος.

Τι αντιπροσωπεύει ο κάθε όρος;

Παρατηρήστε τις περιοδικές συνθήκες που έχουν εισαχθεί.

In [ ]:
for n in range(Nt - 1):

    u[n+1, 1:-1, 1:-1] = (
        u[n, 1:-1, 1:-1] +
        k*Du*(
            u[n, 1:-1, 0:-2] + 
            u[n, 0:-2, 1:-1] - 
            4*u[n, 1:-1, 1:-1] + 
            u[n, 2:, 1:-1] +
            u[n, 1:-1, 2:]
        ) -
        k*u[n, 1:-1, 1:-1]*(v[n, 1:-1, 1:-1]**2) +
        k*F*(1 - u[n, 1:-1, 1:-1])
    )

    v[n+1, 1:-1, 1:-1] = (
        v[n, 1:-1, 1:-1] +
        k*Dv*(
            v[n, 1:-1, 0:-2] + 
            v[n, 0:-2, 1:-1] - 
            4*v[n, 1:-1, 1:-1] + 
            v[n, 2:, 1:-1] +
            v[n, 1:-1, 2:]
        ) +
        k*u[n, 1:-1, 1:-1]*(v[n, 1:-1, 1:-1]**2) -
        k*(F + K) * v[n, 1:-1, 1:-1]
    )
    u[n+1, 0, :] = u[n, -2, :]
    u[n+1, -1, :] = u[n, 1, :]
    u[n+1, :, 0] = u[n, :, -2]
    u[n+1, :, -1] = u[n, :, 1]

    v[n+1, 0, :] = v[n, -2, :]
    v[n+1, -1, :] = v[n, 1, :]
    v[n+1, :, 0] = v[n, :, -2]
    v[n+1, :, -1] = v[n, :, 1]

#### Αναδειγματοληψία ανά 50 χρονικές στιγμές 

(για μείωση του όγκου των δεδομένων)

In [ ]:
step_for_t_subset = 50

u_subset = u[::step_for_t_subset, :, :]
v_subset = v[::step_for_t_subset, :, :]

### Animation της αριθμητικής λύσης (μεταβλητή $u$)

In [ ]:
fig, ax = plt.subplots()
plt.close()

colormap = plt.get_cmap("viridis")


# δημιουργία σταθερού colorbar
img = ax.imshow(np.zeros((1,1)), vmin=0 , vmax=1, cmap=colormap)
fig.colorbar(img, ax=ax, fraction=0.025)

def animate(i):
    ax.clear()

    ax.imshow(u_subset[i, :, :].T, vmin=0, vmax=1, cmap=colormap)

    ax.set_title(f"Time: {t[i]:.2f} seconds", fontweight="bold", loc="center")

ani = FuncAnimation(
    fig=fig,
    func=animate,
    frames=u_subset.shape[0],
    interval=100,
    repeat=False,
)
plt.close()
ani

Δοκιμάστε στη συνέχεια τις παρακάτω αρχικές παραμέτρους, τι μοτίβα προκύπτουν;

### 1.

- $D_u = 0.14$
- $D_v = 0.06$
- $F = 0.035$
- $K = 0.065$


### 2.

- $D_u = 0.16$
- $D_v = 0.08$
- $F = 0.060$
- $K = 0.062$


### 3.

- $D_u = 0.1$
- $D_v = 0.05$
- $F = 0.0545$
- $K = 0.062$
